In [ ]:
pip install transformers

In [ ]:
pip install datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00


In [ ]:
pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.1 MB/s eta 0:00:00


In [ ]:
pip install pandas


In [ ]:
pip install torch


In [ ]:
pip install transformers datasets sacrebleu scikit-learn pandas torch


In [ ]:
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MarianMTModel, MarianTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sacrebleu import corpus_bleu
import pandas as pd
import torch

# Load the IITB English-Hindi dataset
ds = load_dataset("cfilt/iitb-english-hindi")
df = pd.DataFrame(ds['train'])  # Using the training split for this example

# Filter to use only English-Hindi pairs (already provided in the dataset)
df = df[['translation']]

# Create a dataframe with English and Hindi columns
df['English_Text'] = df['translation'].apply(lambda x: x['en'])
df['Hindi_True'] = df['translation'].apply(lambda x: x['hi'])

# For performance reasons, let's limit the number of sentences
df = df.head(100)  # You can modify the number of rows to adjust dataset size

# Define functions to translate English to Hindi using mBART and MarianMT

# 1. Using mBART for translation (English to Hindi)
def translate_with_mbart(text, tgt_lang="hi_IN"):
    try:
        # Load the mBART model and tokenizer for 50 languages
        model_name = 'facebook/mbart-large-50-many-to-many-mmt'
        tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
        model = MBartForConditionalGeneration.from_pretrained(model_name)

        # Set the source language to English
        tokenizer.src_lang = "en_XX"

        # Tokenize the input text and encode it for mBART
        inputs = tokenizer(text, return_tensors="pt")

        # Translate to the target language
        model_inputs = inputs['input_ids']
        generated_tokens = model.generate(model_inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])

        # Decode the tokens to get the translated sentence
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translation
    except Exception as e:
        print(f"Error in mBART translation: {e}")
        return None


# 2. Using MarianMT for translation (English to Hindi)
def translate_with_marian(text, src_lang="en", tgt_lang="hi"):
    try:
        model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

        # Tokenize input text and translate
        batch = tokenizer(text, return_tensors="pt")
        generated_tokens = model.generate(**batch)

        # Decode the translated sentence
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translation
    except Exception as e:
        print(f"Error in Marian translation: {e}")
        return None


# Apply translations to the dataset
df['mBART_Hindi'] = df['English_Text'].apply(
    lambda x: translate_with_mbart(x, tgt_lang="hi_IN")
)

df['Marian_Hindi'] = df['English_Text'].apply(
    lambda x: translate_with_marian(x, src_lang="en", tgt_lang="hi")
)

# Save the dataset with translations to a new CSV file
translated_csv_filename = 'iitb_english_to_hindi_translations.csv'
df.to_csv(translated_csv_filename, index=False)

print(f"Translations added and saved to '{translated_csv_filename}'.")

# Evaluation Metrics

# Function to calculate BLEU score for both models
def calculate_bleu_score(true_texts, predicted_texts):
    bleu = corpus_bleu(predicted_texts, [true_texts])
    return bleu.score

# Calculate BLEU score for mBART
bleu_mbart = calculate_bleu_score(df['Hindi_True'].tolist(), df['mBART_Hindi'].tolist())

# Calculate BLEU score for MarianMT
bleu_marian = calculate_bleu_score(df['Hindi_True'].tolist(), df['Marian_Hindi'].tolist())

print(f"BLEU Score for mBART: {bleu_mbart}")
print(f"BLEU Score for MarianMT: {bleu_marian}")

# Since accuracy, precision, recall, and F1 score are not typical metrics for translation
# we’ll instead focus on BLEU and provide an additional analysis metric such as token-based matching.

# Optional: Token-based Accuracy and F1 Score (matching tokens)

def token_based_evaluation(true_texts, predicted_texts):
    # Tokenize Hindi sentences into words
    true_tokens = [set(text.split()) for text in true_texts]
    predicted_tokens = [set(text.split()) for text in predicted_texts]

    # Calculate accuracy as percentage of tokens matched
    correct_tokens = sum([len(true & pred) for true, pred in zip(true_tokens, predicted_tokens)])
    total_tokens = sum([len(true) for true in true_tokens])

    # Token-based Accuracy
    accuracy = correct_tokens / total_tokens

    return accuracy

# Calculate token-based accuracy
accuracy_mbart = token_based_evaluation(df['Hindi_True'].tolist(), df['mBART_Hindi'].tolist())
accuracy_marian = token_based_evaluation(df['Hindi_True'].tolist(), df['Marian_Hindi'].tolist())

print(f"Token-based Accuracy for mBART: {accuracy_mbart}")
print(f"Token-based Accuracy for MarianMT: {accuracy_marian}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Translations added and saved to 'iitb_english_to_hindi_translations.csv'.
BLEU Score for mBART: 16.656580821775215
BLEU Score for MarianMT: 85.27599415321126
Token-based Accuracy for mBART: 0.3416149068322981
Token-based Accuracy for MarianMT: 0.8788819875776398


In [ ]:
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MarianMTModel, MarianTokenizer
from sklearn.metrics import accuracy_score
from sacrebleu import corpus_bleu
import pandas as pd

# Load the Urdu-English dataset
ds = load_dataset("HaiderSultanArc/MT-Urdu-English")

# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(ds['train'])  # Using the training split for this example

# Check the actual columns in the DataFrame
print(df.columns)  # Check the column names

# Assuming the dataset has 'en' and 'ur' columns for English and Urdu
df = df[['en', 'ur']]  # Keep only the relevant columns
df.columns = ['english', 'urdu']  # Rename columns for clarity

# For performance reasons, let's limit the number of sentences
df = df.head(100)  # You can increase the dataset size as needed

# Define functions to translate English to Urdu using mBART and MarianMT

# 1. Using mBART for translation (English to Urdu)
def translate_with_mbart(text, tgt_lang="ur_PK"):
    try:
        # Load the mBART model and tokenizer for 50 languages
        model_name = 'facebook/mbart-large-50-many-to-many-mmt'
        tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
        model = MBartForConditionalGeneration.from_pretrained(model_name)

        # Set the source language to English
        tokenizer.src_lang = "en_XX"

        # Tokenize the input text and encode it for mBART
        inputs = tokenizer(text, return_tensors="pt")

        # Translate to the target language
        model_inputs = inputs['input_ids']
        generated_tokens = model.generate(model_inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])

        # Decode the tokens to get the translated sentence
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translation
    except Exception as e:
        print(f"Error in mBART translation: {e}")
        return ""

# 2. Using MarianMT for translation (English to Urdu)
def translate_with_marian(text, src_lang="en", tgt_lang="ur"):
    try:
        model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'  # Correct model name for English to Urdu
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

        # Tokenize input text and translate
        batch = tokenizer(text, return_tensors="pt")
        generated_tokens = model.generate(**batch)

        # Decode the translated sentence
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translation
    except Exception as e:
        print(f"Error in Marian translation: {e}")
        return ""

# Apply translations to the dataset
df['mBART_Urdu'] = df['english'].apply(
    lambda x: translate_with_mbart(x, tgt_lang="ur_PK")
)

df['Marian_Urdu'] = df['english'].apply(
    lambda x: translate_with_marian(x, src_lang="en", tgt_lang="ur")
)

# Save the dataset with translations to a new CSV file
translated_csv_filename = 'english_to_urdu_translations.csv'
df.to_csv(translated_csv_filename, index=False)

print(f"Translations added and saved to '{translated_csv_filename}'.")

# Evaluation Metrics

# Function to calculate BLEU score for both models
def calculate_bleu_score(true_texts, predicted_texts):
    # Filter out empty translations
    valid_true_texts = [true for true, pred in zip(true_texts, predicted_texts) if pred]
    valid_predicted_texts = [pred for pred in predicted_texts if pred]

    # Ensure inputs are not empty
    if valid_true_texts and valid_predicted_texts:
        bleu = corpus_bleu(valid_predicted_texts, [valid_true_texts])
        return bleu.score
    else:
        return 0

# Calculate BLEU score for mBART
bleu_mbart = calculate_bleu_score(df['urdu'].tolist(), df['mBART_Urdu'].tolist())

# Calculate BLEU score for MarianMT
bleu_marian = calculate_bleu_score(df['urdu'].tolist(), df['Marian_Urdu'].tolist())

print(f"BLEU Score for mBART (English to Urdu): {bleu_mbart}")
print(f"BLEU Score for MarianMT (English to Urdu): {bleu_marian}")

# Optional: Token-based Accuracy and F1 Score (matching tokens)

def token_based_evaluation(true_texts, predicted_texts):
    # Tokenize Urdu sentences into words
    true_tokens = [set(text.split()) for text in true_texts]
    predicted_tokens = [set(text.split()) for text in predicted_texts]

    # Calculate accuracy as percentage of tokens matched
    correct_tokens = sum([len(true & pred) for true, pred in zip(true_tokens, predicted_tokens)])
    total_tokens = sum([len(true) for true in true_tokens])

    # Token-based Accuracy
    accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0

    return accuracy

# Calculate token-based accuracy
accuracy_mbart = token_based_evaluation(df['urdu'].tolist(), df['mBART_Urdu'].tolist())
accuracy_marian = token_based_evaluation(df['urdu'].tolist(), df['Marian_Urdu'].tolist())

print(f"Token-based Accuracy for mBART (English to Urdu): {accuracy_mbart}")
print(f"Token-based Accuracy for MarianMT (English to Urdu): {accuracy_marian}")


Index(['en', 'ur'], dtype='object')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Translations added and saved to 'english_to_urdu_translations.csv'.
BLEU Score for mBART (English to Urdu): 5.0243112714751
BLEU Score for MarianMT (English to Urdu): 8.463973455936678
Token-based Accuracy for mBART (English to Urdu): 0.21803977272727273
Token-based Accuracy for MarianMT (English to Urdu): 0.2975852272727273


In [ ]:
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MarianMTModel, MarianTokenizer
from sklearn.metrics import accuracy_score
from sacrebleu import corpus_bleu
import pandas as pd
import torch

# Load the English to Malayalam dataset
ds = load_dataset("Hemanth-thunder/english-to-malayalam-mt")
df = pd.DataFrame(ds['train'])  # Using the training split for this example

# Assuming the dataset has 'en' and 'ml' columns for English and Malayalam
df = df[['en', 'ml']]  # Keep only the relevant columns

# Limit the number of sentences for performance reasons
df = df.head(100)  # Modify this to test with more data if needed

# Define functions to translate English to Malayalam using mBART and MarianMT

# 1. Using mBART for translation (English to Malayalam)
def translate_with_mbart(text, tgt_lang="ml_IN"):
    try:
        # Load the mBART model and tokenizer for 50 languages
        model_name = 'facebook/mbart-large-50-many-to-many-mmt'
        tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
        model = MBartForConditionalGeneration.from_pretrained(model_name)

        # Set the source language to English
        tokenizer.src_lang = "en_XX"

        # Tokenize the input text and encode it for mBART (with max_length set)
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

        # Translate to the target language
        model_inputs = inputs['input_ids']
        generated_tokens = model.generate(model_inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])

        # Decode the tokens to get the translated sentence
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translation
    except Exception as e:
        print(f"Error in mBART translation: {e}")
        return ""

# 2. Using MarianMT for translation (English to Malayalam)
def translate_with_marian(text, src_lang="en", tgt_lang="ml"):
    try:
        model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'  # Correct model name for English to Malayalam
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

        # Tokenize input text and translate (with max_length set)
        batch = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        generated_tokens = model.generate(**batch)

        # Decode the translated sentence
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translation
    except Exception as e:
        print(f"Error in Marian translation: {e}")
        return ""

# Apply translations to the dataset
df['mBART_Malayalam'] = df['en'].apply(lambda x: translate_with_mbart(x, tgt_lang="ml_IN"))
df['Marian_Malayalam'] = df['en'].apply(lambda x: translate_with_marian(x, src_lang="en", tgt_lang="ml"))

# Save the dataset with translations to a new CSV file
translated_csv_filename = 'english_to_malayalam_translations.csv'
df.to_csv(translated_csv_filename, index=False)

print(f"Translations added and saved to '{translated_csv_filename}'.")

# Evaluation Metrics

# Function to calculate BLEU score for both models
def calculate_bleu_score(true_texts, predicted_texts):
    # Filter out empty translations
    valid_true_texts = [true for true, pred in zip(true_texts, predicted_texts) if pred]
    valid_predicted_texts = [pred for pred in predicted_texts if pred]

    # Ensure inputs are not empty
    if valid_true_texts and valid_predicted_texts:
        bleu = corpus_bleu(valid_predicted_texts, [valid_true_texts])
        return bleu.score
    else:
        return 0

# Calculate BLEU score for mBART
bleu_mbart = calculate_bleu_score(df['ml'].tolist(), df['mBART_Malayalam'].tolist())

# Calculate BLEU score for MarianMT
bleu_marian = calculate_bleu_score(df['ml'].tolist(), df['Marian_Malayalam'].tolist())

print(f"BLEU Score for mBART (English to Malayalam): {bleu_mbart}")
print(f"BLEU Score for MarianMT (English to Malayalam): {bleu_marian}")

# Optional: Token-based Accuracy and F1 Score (matching tokens)
def token_based_evaluation(true_texts, predicted_texts):
    # Tokenize Malayalam sentences into words
    true_tokens = [set(text.split()) for text in true_texts]
    predicted_tokens = [set(text.split()) for text in predicted_texts]

    # Calculate accuracy as percentage of tokens matched
    correct_tokens = sum([len(true & pred) for true, pred in zip(true_tokens, predicted_tokens)])
    total_tokens = sum([len(true) for true in true_tokens])

    # Token-based Accuracy
    accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0

    return accuracy

# Calculate token-based accuracy
accuracy_mbart = token_based_evaluation(df['ml'].tolist(), df['mBART_Malayalam'].tolist())
accuracy_marian = token_based_evaluation(df['ml'].tolist(), df['Marian_Malayalam'].tolist())

print(f"Token-based Accuracy for mBART (English to Malayalam): {accuracy_mbart}")
print(f"Token-based Accuracy for MarianMT (English to Malayalam): {accuracy_marian}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/449k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/614k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/956k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/229M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Translations added and saved to 'english_to_malayalam_translations.csv'.
BLEU Score for mBART (English to Malayalam): 2.1080599669072426
BLEU Score for MarianMT (English to Malayalam): 6.301880857486158
Token-based Accuracy for mBART (English to Malayalam): 0.10510510510510511
Token-based Accuracy for MarianMT (English to Malayalam): 0.17117117117117117
